In [46]:
import pandas as pd
import numpy as np

# Load dim
target_path = 'Data/Target/'

dim_customer = pd.read_csv(target_path + 'DimCustomer.csv')
dim_city = pd.read_csv(target_path + 'DimCity.csv')
dim_date = pd.read_csv(target_path + 'DimDate.csv')
dim_orderStatus = pd.read_csv(target_path + 'DimOrderStatus.csv')
dim_product = pd.read_csv(target_path + 'DimProduct.csv')
dim_returnReason = pd.read_csv(target_path + 'DimReturnReason.csv')
dim_salesAgent = pd.read_csv(target_path + 'DimSalesAgent.csv') 
dim_state = pd.read_csv(target_path + 'DimState.csv')

dim_date['DateKey'] = dim_date['DateKey'].astype(str)

#load original data
dataset_path = r'Data\Original\Dataset.xlsx'
dict_df = pd.read_excel(dataset_path, 
                            sheet_name=[i for i in range(3)])

sales = pd.concat([dict_df.get(i) for i in range(len(dict_df))])
sales.shape


(3000, 14)

In [47]:
sales['Return Reason'].replace(np.nan, 'None', inplace=True)
sales.head()

C:\Users\TOAN.000\AppData\Local\Temp\ipykernel_21460\2368643829.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sales['Return Reason'].replace(np.nan, 'None', inplace=True)


,Order ID,Customer Last Name,Customer First Name,Customer ID,Product Category,Order Date,Quantity Ordered,Order Status,State,Manufacturer Price,Sale Price,Total Profit (GMROI),Sales Agent,Return Reason
0,1001,Hughes,Lewis,1234,Clothing,2022-10-11,16,Delivered,LA,1485,1880,395,Kelly,None
1,1002,Gomez,Ali,1235,Home & Kitchen,2022-10-09,11,Delivered,LA,1310,1602,292,Brian,None
2,1003,Beck,Mila,1236,Personal Care,2022-10-21,4,Shipped,GA,800,891,91,Terrence,None
3,1004,Juarez,Ryleigh,1237,Personal Care,2022-10-26,20,Cancelled,NC,1190,1462,272,Brian,Defective
4,1005,Preston,Leonidas,1238,Electronics,2022-10-22,9,Shipped,GA,470,525,55,Daniel,None


In [52]:
fact_df = pd.merge(sales, dim_customer['CustomerKey'], 
                   how='left', 
                   left_on='Customer ID', 
                   right_on='CustomerKey').drop(
                       columns=['Customer ID', 
                                'Customer Last Name', 
                                'Customer First Name'])

fact_df = pd.merge(fact_df, dim_state, 
                   how='left', 
                   left_on='State', 
                   right_on='State').drop(columns=['State'])

fact_df = pd.merge(fact_df, dim_orderStatus, 
                left_on='Order Status', 
                right_on='OrderStatusName').drop(columns=['OrderStatusName', 'Order Status'])

fact_df = pd.merge(fact_df, dim_returnReason,
                   left_on='Return Reason',
                   right_on='ReturnReasonName').drop(columns=['ReturnReasonName', 'Return Reason'])

fact_df = pd.merge(fact_df, dim_salesAgent, 
                   how='left',
                   left_on='Sales Agent',
                   right_on='SaleAgentName').drop(columns=['SaleAgentName', 'Sales Agent'])

fact_df = pd.merge(fact_df, dim_product, 
                left_on='Product Category',
                right_on='ProductCatalogName').drop(columns=['ProductCatalogName', 'Product Category'])

In [53]:
fact_df['Order Date'] = fact_df['Order Date'].apply(lambda x: x.strftime('%Y%m%d'))

fact_df = pd.merge(fact_df, dim_date['DateKey'], 
                left_on='Order Date',
                right_on='DateKey').drop(columns=['Order Date'])

In [54]:
fact_df.head()

,Order ID,Quantity Ordered,Manufacturer Price,Sale Price,Total Profit (GMROI),CustomerKey,StateKey,OrderStatusKey,ReturnReasonKey,SaleAgentKey,ProductCatalogKey,DateKey
0,1004,20,1190,1462,272,1237,3,3,2,2,3,20221026
1,1021,14,320,412,92,1254,5,3,2,4,7,20221017
2,1021,14,320,412,92,1254,5,3,2,4,7,20221017
3,1029,6,80,89,9,1262,6,3,3,1,5,20221003
4,1029,6,80,89,9,1262,6,3,3,1,5,20221003


In [55]:
fact_df.to_csv(target_path + 'FactOrder.csv', index=False)